# `customer_usage_<varianto_nr>` failo agregavimas pagal vartotojo id

Šiame Jupyter bloknote pateiktas paprasto agregavimo pavyzdys. Tarpiniai rezultatai išvedami į ekraną, todėl bloknotas sąlyginai ilgas, nors visas reikalingas kodas pilnai tilptų į dalį ekrano aukščio.

In [35]:
import os

import pyspark

In [36]:
spark = (
    pyspark.sql.SparkSession
    .builder
    .appName("Python Spark SQL aggregation example")
    .enableHiveSupport()
    .getOrCreate()
)

Sukursime direktoriją visiems tarpiniams duomenų failams ir kitoms išvestims.

In [37]:
os.makedirs(os.path.join(os.getcwd(), "..", "data", "output"), exist_ok=True)

In [38]:
! tree ../data

../data
├── create_data_sample.sh
├── external
├── interim
│   ├── customer_churn.csv
│   └── customer_usage.csv
├── output
│   ├── aggregated_customer_usage
│   │   ├── part-00000-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00001-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00002-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00003-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00004-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00005-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00006-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00007-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00008-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00009-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00010-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── part-00011-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
│   │   ├── pa

### CSV Nuskaitymas

In [39]:
usage_df = spark.read.csv("../data/interim/customer_usage.csv", header=True, inferSchema=True)

In [40]:
usage_df

DataFrame[year: int, month: int, user_account_id: int, user_lifetime: int, user_intake: int, user_no_outgoing_activity_in_days: int, user_account_balance_last: double, user_spendings: double, user_has_outgoing_calls: int, user_has_outgoing_sms: int, user_use_gprs: int, user_does_reload: int, reloads_inactive_days: int, reloads_count: int, reloads_sum: double, calls_outgoing_count: int, calls_outgoing_spendings: double, calls_outgoing_duration: double, calls_outgoing_spendings_max: double, calls_outgoing_duration_max: double, calls_outgoing_inactive_days: int, calls_outgoing_to_onnet_count: int, calls_outgoing_to_onnet_spendings: double, calls_outgoing_to_onnet_duration: double, calls_outgoing_to_onnet_inactive_days: int, calls_outgoing_to_offnet_count: int, calls_outgoing_to_offnet_spendings: double, calls_outgoing_to_offnet_duration: double, calls_outgoing_to_offnet_inactive_days: int, calls_outgoing_to_abroad_count: int, calls_outgoing_to_abroad_spendings: double, calls_outgoing_to_a

Sukuriame laikiną lentelę.

In [41]:
usage_df.createOrReplaceTempView("customer_usage")

Datos kintamieji.

In [42]:
date_columns = ["year", "month"]

Vartotojo ID kintamasis, saugome `list`'e , kad būtų paprasčiau apjunti vėliau.

In [43]:
id_columns = ["user_account_id"]

Binariniai kintamieji.

In [44]:
binary_columns = [
    "user_intake",
    "user_has_outgoing_calls", "user_has_outgoing_sms", 
    "user_use_gprs", "user_does_reload"
]

Sukuriame tolydžiūjų kintamųjų `list`'ą.

In [45]:
categorical_columns = date_columns + binary_columns + id_columns

continuous_columns = [c for c in usage_df.columns if c not in categorical_columns]
continuous_columns

['user_lifetime',
 'user_no_outgoing_activity_in_days',
 'user_account_balance_last',
 'user_spendings',
 'reloads_inactive_days',
 'reloads_count',
 'reloads_sum',
 'calls_outgoing_count',
 'calls_outgoing_spendings',
 'calls_outgoing_duration',
 'calls_outgoing_spendings_max',
 'calls_outgoing_duration_max',
 'calls_outgoing_inactive_days',
 'calls_outgoing_to_onnet_count',
 'calls_outgoing_to_onnet_spendings',
 'calls_outgoing_to_onnet_duration',
 'calls_outgoing_to_onnet_inactive_days',
 'calls_outgoing_to_offnet_count',
 'calls_outgoing_to_offnet_spendings',
 'calls_outgoing_to_offnet_duration',
 'calls_outgoing_to_offnet_inactive_days',
 'calls_outgoing_to_abroad_count',
 'calls_outgoing_to_abroad_spendings',
 'calls_outgoing_to_abroad_duration',
 'calls_outgoing_to_abroad_inactive_days',
 'sms_outgoing_count',
 'sms_outgoing_spendings',
 'sms_outgoing_spendings_max',
 'sms_outgoing_inactive_days',
 'sms_outgoing_to_onnet_count',
 'sms_outgoing_to_onnet_spendings',
 'sms_outgoing

Sukuriame vidurkių SQL išraikškas tolydiesiems kintamiesiems.

In [46]:
sql_expressions_avg = ["AVG({0}) AS {0}".format(c) for c in continuous_columns]
sql_expressions_avg

['AVG(user_lifetime) AS user_lifetime',
 'AVG(user_no_outgoing_activity_in_days) AS user_no_outgoing_activity_in_days',
 'AVG(user_account_balance_last) AS user_account_balance_last',
 'AVG(user_spendings) AS user_spendings',
 'AVG(reloads_inactive_days) AS reloads_inactive_days',
 'AVG(reloads_count) AS reloads_count',
 'AVG(reloads_sum) AS reloads_sum',
 'AVG(calls_outgoing_count) AS calls_outgoing_count',
 'AVG(calls_outgoing_spendings) AS calls_outgoing_spendings',
 'AVG(calls_outgoing_duration) AS calls_outgoing_duration',
 'AVG(calls_outgoing_spendings_max) AS calls_outgoing_spendings_max',
 'AVG(calls_outgoing_duration_max) AS calls_outgoing_duration_max',
 'AVG(calls_outgoing_inactive_days) AS calls_outgoing_inactive_days',
 'AVG(calls_outgoing_to_onnet_count) AS calls_outgoing_to_onnet_count',
 'AVG(calls_outgoing_to_onnet_spendings) AS calls_outgoing_to_onnet_spendings',
 'AVG(calls_outgoing_to_onnet_duration) AS calls_outgoing_to_onnet_duration',
 'AVG(calls_outgoing_to_onne

Sukuriame maksimumo SQL išraiškas binariniams kintamiesiems.

In [47]:
sql_expressions_max = ["MAX({0}) AS {0}".format(c) for c in binary_columns]
sql_expressions_max

['MAX(user_intake) AS user_intake',
 'MAX(user_has_outgoing_calls) AS user_has_outgoing_calls',
 'MAX(user_has_outgoing_sms) AS user_has_outgoing_sms',
 'MAX(user_use_gprs) AS user_use_gprs',
 'MAX(user_does_reload) AS user_does_reload']

Sukuriame mėnesių [vartotojo išrašų] skaičiavimo SQL išraišką.

In [48]:
sql_expressions_count = ["COUNT(*) AS n_months"]
sql_expressions_count

['COUNT(*) AS n_months']

Apjungiame visas SQL išraiškas.

In [49]:
sql_expressions_aggregation = sql_expressions_avg + sql_expressions_max + sql_expressions_count
sql_expressions_aggregation

['AVG(user_lifetime) AS user_lifetime',
 'AVG(user_no_outgoing_activity_in_days) AS user_no_outgoing_activity_in_days',
 'AVG(user_account_balance_last) AS user_account_balance_last',
 'AVG(user_spendings) AS user_spendings',
 'AVG(reloads_inactive_days) AS reloads_inactive_days',
 'AVG(reloads_count) AS reloads_count',
 'AVG(reloads_sum) AS reloads_sum',
 'AVG(calls_outgoing_count) AS calls_outgoing_count',
 'AVG(calls_outgoing_spendings) AS calls_outgoing_spendings',
 'AVG(calls_outgoing_duration) AS calls_outgoing_duration',
 'AVG(calls_outgoing_spendings_max) AS calls_outgoing_spendings_max',
 'AVG(calls_outgoing_duration_max) AS calls_outgoing_duration_max',
 'AVG(calls_outgoing_inactive_days) AS calls_outgoing_inactive_days',
 'AVG(calls_outgoing_to_onnet_count) AS calls_outgoing_to_onnet_count',
 'AVG(calls_outgoing_to_onnet_spendings) AS calls_outgoing_to_onnet_spendings',
 'AVG(calls_outgoing_to_onnet_duration) AS calls_outgoing_to_onnet_duration',
 'AVG(calls_outgoing_to_onne

SQL SELECT išraiškas įstatome į SQL užklausą ir ją įvykdome.

In [50]:
sql_query_aggregate_by_user_id = """
SELECT user_account_id, {}
FROM customer_usage
GROUP BY user_account_id
""".format("\n , ".join(sql_expressions_aggregation))

print(sql_query_aggregate_by_user_id)


SELECT user_account_id, AVG(user_lifetime) AS user_lifetime
 , AVG(user_no_outgoing_activity_in_days) AS user_no_outgoing_activity_in_days
 , AVG(user_account_balance_last) AS user_account_balance_last
 , AVG(user_spendings) AS user_spendings
 , AVG(reloads_inactive_days) AS reloads_inactive_days
 , AVG(reloads_count) AS reloads_count
 , AVG(reloads_sum) AS reloads_sum
 , AVG(calls_outgoing_count) AS calls_outgoing_count
 , AVG(calls_outgoing_spendings) AS calls_outgoing_spendings
 , AVG(calls_outgoing_duration) AS calls_outgoing_duration
 , AVG(calls_outgoing_spendings_max) AS calls_outgoing_spendings_max
 , AVG(calls_outgoing_duration_max) AS calls_outgoing_duration_max
 , AVG(calls_outgoing_inactive_days) AS calls_outgoing_inactive_days
 , AVG(calls_outgoing_to_onnet_count) AS calls_outgoing_to_onnet_count
 , AVG(calls_outgoing_to_onnet_spendings) AS calls_outgoing_to_onnet_spendings
 , AVG(calls_outgoing_to_onnet_duration) AS calls_outgoing_to_onnet_duration
 , AVG(calls_outgoing_

In [51]:
aggregate_usage_df = spark.sql(sql_query_aggregate_by_user_id)

In [52]:
aggregate_usage_df.first()

Row(user_account_id=978177, user_lifetime=289.0, user_no_outgoing_activity_in_days=2.6666666666666665, user_account_balance_last=0.5, user_spendings=2.0, reloads_inactive_days=33.666666666666664, reloads_count=0.6666666666666666, reloads_sum=2.5, calls_outgoing_count=40.0, calls_outgoing_spendings=1.75, calls_outgoing_duration=9.15, calls_outgoing_spendings_max=0.73, calls_outgoing_duration_max=2.893333333333333, calls_outgoing_inactive_days=2.6666666666666665, calls_outgoing_to_onnet_count=0.0, calls_outgoing_to_onnet_spendings=0.0, calls_outgoing_to_onnet_duration=0.0, calls_outgoing_to_onnet_inactive_days=2.6666666666666665, calls_outgoing_to_offnet_count=6.0, calls_outgoing_to_offnet_spendings=0.5466666666666666, calls_outgoing_to_offnet_duration=3.65, calls_outgoing_to_offnet_inactive_days=2.6666666666666665, calls_outgoing_to_abroad_count=3.0, calls_outgoing_to_abroad_spendings=0.13999999999999999, calls_outgoing_to_abroad_duration=0.49, calls_outgoing_to_abroad_inactive_days=2.6

Išsaugome agreguotus pagal vartojoją duomenis.

In [54]:
aggregate_usage_df.write.csv("../data/output/aggregated_customer_usage")

AnalysisException: 'path file:/home/majo/Dropbox/Studijos/Magistras/Didziuju duomenu rinkiniu tyrybos metodai/Projektas/p160m132-projektas/data/output/aggregated_customer_usage already exists.;'

In [27]:
! cat ../data/output/aggregated_customer_usage/* | head -n 5

978177,289.0,2.6666666666666665,0.5,2.0,33.666666666666664,0.6666666666666666,2.5,40.0,1.75,9.15,0.73,2.893333333333333,2.6666666666666665,0.0,0.0,0.0,2.6666666666666665,6.0,0.5466666666666666,3.65,2.6666666666666665,3.0,0.13999999999999999,0.49,2.6666666666666665,3.3333333333333335,0.25,0.03666666666666667,38.333333333333336,0.0,0.0,38.333333333333336,2.3333333333333335,0.13999999999999999,38.333333333333336,0.0,0.0,38.333333333333336,8.666666666666666,0.0,1.6666666666666667,0.0,0.0,0.0,0.0,1307.0,1.6666666666666667,8.5,34.1,0.0,8.633333333333333,5.023333333333334,19.333333333333332,0.0,14.666666666666666,2.6666666666666665,0.0,0,1,1,0,1,3
28024,955.0,1.6666666666666667,5.246666666666667,20.896666666666665,9.0,3.6666666666666665,22.02,71.66666666666667,20.896666666666665,393.24333333333334,2.0766666666666667,57.333333333333336,3.3333333333333335,0.0,0.0,0.0,3.3333333333333335,24.0,12.926666666666668,317.0,3.3333333333333335,0.0,0.0,0.0,3.3333333333333335,29.333333333333332,0.0,0.0,2.0

In [28]:
! ls ../data/output/aggregated_customer_usage/

part-00000-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00001-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00002-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00003-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00004-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00005-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00006-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00007-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00008-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00009-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00010-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00011-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00012-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00013-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00014-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00015-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00016-4bb548db-468c-44b8-a839-59769b2cb5b6-c000.csv
part-00017-4bb548db-468c-44b8-a

Kadangi duomenys išsagojami į direktoriją po failą kiekvienai Spark duomenų particijai, atskirai išsaugojame stulpelių pavadinimus.

In [55]:
with open("../data/output/header__aggregated_customer_usage.txt", "w") as f:
    f.write(",".join(aggregate_usage_df.columns) + "\n")

In [56]:
! cat ../data/output/header__aggregated_customer_usage.txt

user_account_id,user_lifetime,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,reloads_inactive_days,reloads_count,reloads_sum,calls_outgoing_count,calls_outgoing_spendings,calls_outgoing_duration,calls_outgoing_spendings_max,calls_outgoing_duration_max,calls_outgoing_inactive_days,calls_outgoing_to_onnet_count,calls_outgoing_to_onnet_spendings,calls_outgoing_to_onnet_duration,calls_outgoing_to_onnet_inactive_days,calls_outgoing_to_offnet_count,calls_outgoing_to_offnet_spendings,calls_outgoing_to_offnet_duration,calls_outgoing_to_offnet_inactive_days,calls_outgoing_to_abroad_count,calls_outgoing_to_abroad_spendings,calls_outgoing_to_abroad_duration,calls_outgoing_to_abroad_inactive_days,sms_outgoing_count,sms_outgoing_spendings,sms_outgoing_spendings_max,sms_outgoing_inactive_days,sms_outgoing_to_onnet_count,sms_outgoing_to_onnet_spendings,sms_outgoing_to_onnet_inactive_days,sms_outgoing_to_offnet_count,sms_outgoing_to_offnet_spendings,sms_outgoing_to_offnet_i

Kas bus, jeigu duomenis nuskaitysime be stulpelių pavadinimų:

Nuskaitome stulpelių pavadinimus ir sukuriame jų `list`'ą

In [57]:
with open("../data/output/header__aggregated_customer_usage.txt") as f:
    columns = f.read().rstrip().split(",")

In [58]:
columns

['user_account_id',
 'user_lifetime',
 'user_no_outgoing_activity_in_days',
 'user_account_balance_last',
 'user_spendings',
 'reloads_inactive_days',
 'reloads_count',
 'reloads_sum',
 'calls_outgoing_count',
 'calls_outgoing_spendings',
 'calls_outgoing_duration',
 'calls_outgoing_spendings_max',
 'calls_outgoing_duration_max',
 'calls_outgoing_inactive_days',
 'calls_outgoing_to_onnet_count',
 'calls_outgoing_to_onnet_spendings',
 'calls_outgoing_to_onnet_duration',
 'calls_outgoing_to_onnet_inactive_days',
 'calls_outgoing_to_offnet_count',
 'calls_outgoing_to_offnet_spendings',
 'calls_outgoing_to_offnet_duration',
 'calls_outgoing_to_offnet_inactive_days',
 'calls_outgoing_to_abroad_count',
 'calls_outgoing_to_abroad_spendings',
 'calls_outgoing_to_abroad_duration',
 'calls_outgoing_to_abroad_inactive_days',
 'sms_outgoing_count',
 'sms_outgoing_spendings',
 'sms_outgoing_spendings_max',
 'sms_outgoing_inactive_days',
 'sms_outgoing_to_onnet_count',
 'sms_outgoing_to_onnet_spendi

Nuskaitome duomenis, sudarome python dict'ą iš esamų stulpelių pavadinimų į tikruosius pavadinimus, ir pervadiname stulpepių pavadinimus

In [30]:
aggregate_usage_without_columns_df = (
    spark.read.csv("../data/output/aggregated_customer_usage/", inferSchema=True, header=False)
)

In [31]:
default_column_names = aggregate_usage_without_columns_df.columns
default_column_names

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15',
 '_c16',
 '_c17',
 '_c18',
 '_c19',
 '_c20',
 '_c21',
 '_c22',
 '_c23',
 '_c24',
 '_c25',
 '_c26',
 '_c27',
 '_c28',
 '_c29',
 '_c30',
 '_c31',
 '_c32',
 '_c33',
 '_c34',
 '_c35',
 '_c36',
 '_c37',
 '_c38',
 '_c39',
 '_c40',
 '_c41',
 '_c42',
 '_c43',
 '_c44',
 '_c45',
 '_c46',
 '_c47',
 '_c48',
 '_c49',
 '_c50',
 '_c51',
 '_c52',
 '_c53',
 '_c54',
 '_c55',
 '_c56',
 '_c57',
 '_c58',
 '_c59',
 '_c60',
 '_c61',
 '_c62',
 '_c63']

In [32]:
aggregate_usage_columns_map = dict(zip(default_column_names, columns))

aggregate_usage_columns_map

{'_c0': 'user_account_id',
 '_c1': 'user_lifetime',
 '_c10': 'calls_outgoing_duration',
 '_c11': 'calls_outgoing_spendings_max',
 '_c12': 'calls_outgoing_duration_max',
 '_c13': 'calls_outgoing_inactive_days',
 '_c14': 'calls_outgoing_to_onnet_count',
 '_c15': 'calls_outgoing_to_onnet_spendings',
 '_c16': 'calls_outgoing_to_onnet_duration',
 '_c17': 'calls_outgoing_to_onnet_inactive_days',
 '_c18': 'calls_outgoing_to_offnet_count',
 '_c19': 'calls_outgoing_to_offnet_spendings',
 '_c2': 'user_no_outgoing_activity_in_days',
 '_c20': 'calls_outgoing_to_offnet_duration',
 '_c21': 'calls_outgoing_to_offnet_inactive_days',
 '_c22': 'calls_outgoing_to_abroad_count',
 '_c23': 'calls_outgoing_to_abroad_spendings',
 '_c24': 'calls_outgoing_to_abroad_duration',
 '_c25': 'calls_outgoing_to_abroad_inactive_days',
 '_c26': 'sms_outgoing_count',
 '_c27': 'sms_outgoing_spendings',
 '_c28': 'sms_outgoing_spendings_max',
 '_c29': 'sms_outgoing_inactive_days',
 '_c3': 'user_account_balance_last',
 '_c30'

In [33]:
aggregate_usage_with_columns_df = (
    aggregate_usage_without_columns_df
    .select([pyspark.sql.functions.col(c).alias(aggregate_usage_columns_map.get(c, c)) 
             for c in default_column_names])
)

In [34]:
aggregate_usage_with_columns_df.columns

['user_account_id',
 'user_lifetime',
 'user_no_outgoing_activity_in_days',
 'user_account_balance_last',
 'user_spendings',
 'reloads_inactive_days',
 'reloads_count',
 'reloads_sum',
 'calls_outgoing_count',
 'calls_outgoing_spendings',
 'calls_outgoing_duration',
 'calls_outgoing_spendings_max',
 'calls_outgoing_duration_max',
 'calls_outgoing_inactive_days',
 'calls_outgoing_to_onnet_count',
 'calls_outgoing_to_onnet_spendings',
 'calls_outgoing_to_onnet_duration',
 'calls_outgoing_to_onnet_inactive_days',
 'calls_outgoing_to_offnet_count',
 'calls_outgoing_to_offnet_spendings',
 'calls_outgoing_to_offnet_duration',
 'calls_outgoing_to_offnet_inactive_days',
 'calls_outgoing_to_abroad_count',
 'calls_outgoing_to_abroad_spendings',
 'calls_outgoing_to_abroad_duration',
 'calls_outgoing_to_abroad_inactive_days',
 'sms_outgoing_count',
 'sms_outgoing_spendings',
 'sms_outgoing_spendings_max',
 'sms_outgoing_inactive_days',
 'sms_outgoing_to_onnet_count',
 'sms_outgoing_to_onnet_spendi

In [35]:
aggregate_usage_with_columns_df.first()

Row(user_account_id=984822, user_lifetime=457.0, user_no_outgoing_activity_in_days=1.0, user_account_balance_last=8.883333333333335, user_spendings=1.9733333333333334, reloads_inactive_days=37.0, reloads_count=0.3333333333333333, reloads_sum=1.3333333333333333, calls_outgoing_count=27.333333333333332, calls_outgoing_spendings=1.6533333333333333, calls_outgoing_duration=19.233333333333334, calls_outgoing_spendings_max=0.3466666666666667, calls_outgoing_duration_max=5.816666666666667, calls_outgoing_inactive_days=1.0, calls_outgoing_to_onnet_count=0.0, calls_outgoing_to_onnet_spendings=0.0, calls_outgoing_to_onnet_duration=0.0, calls_outgoing_to_onnet_inactive_days=1.0, calls_outgoing_to_offnet_count=20.0, calls_outgoing_to_offnet_spendings=1.3033333333333335, calls_outgoing_to_offnet_duration=8.676666666666668, calls_outgoing_to_offnet_inactive_days=1.0, calls_outgoing_to_abroad_count=0.0, calls_outgoing_to_abroad_spendings=0.0, calls_outgoing_to_abroad_duration=0.0, calls_outgoing_to_a

Patogumo dėlei išsauogojame kintamųjų pavadinimus pagal jų tipus.

In [36]:
with open("../data/output/columns_continuous__agg_usage.txt", "w") as f:
    f.write(",".join(continuous_columns) + "\n")

In [37]:
with open("../data/output/columns_binary__agg_usage.txt", "w") as f:
    f.write(",".join(binary_columns) + "\n")

In [38]:
with open("../data/output/columns_ids__agg_usage.txt", "w") as f:
    f.write(",".join(id_columns) + "\n")

In [39]:
with open("../data/output/columns_misc__agg_usage.txt", "w") as f:
    f.write("n_months"  + "\n")

In [40]:
! tree ../data/output

../data/output
├── aggregated_customer_usage
│   ├── part-00000-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00001-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00002-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00003-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00004-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00005-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00006-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00007-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00008-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00009-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00010-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00011-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00012-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00013-ce34a408-a8c4-4148-8da2-cc73e155b005-c000.csv
│   ├── part-00014-ce34a408-a8c4-4148-8da2-cc

Kadangi šiuo konkrečiu atveju dirbame su sąlyginai nedideliu duomenų kiekiu lokalioje failų sistemoje, pasinaudodami Linux komanda `cat` galime apjungti stulpelių pavadinių failą ir iš Apache Spark gautus išvesties failus į vieną ir taip sudaryti`.csv` failą su stulpelių pavadinimais

In [41]:
! cat ../data/output/header__aggregated_customer_usage.txt ../data/output/aggregated_customer_usage/* > ../data/output/aggregated_customer_usage.csv

In [42]:
! head ../data/output/aggregated_customer_usage.csv

user_account_id,user_lifetime,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,reloads_inactive_days,reloads_count,reloads_sum,calls_outgoing_count,calls_outgoing_spendings,calls_outgoing_duration,calls_outgoing_spendings_max,calls_outgoing_duration_max,calls_outgoing_inactive_days,calls_outgoing_to_onnet_count,calls_outgoing_to_onnet_spendings,calls_outgoing_to_onnet_duration,calls_outgoing_to_onnet_inactive_days,calls_outgoing_to_offnet_count,calls_outgoing_to_offnet_spendings,calls_outgoing_to_offnet_duration,calls_outgoing_to_offnet_inactive_days,calls_outgoing_to_abroad_count,calls_outgoing_to_abroad_spendings,calls_outgoing_to_abroad_duration,calls_outgoing_to_abroad_inactive_days,sms_outgoing_count,sms_outgoing_spendings,sms_outgoing_spendings_max,sms_outgoing_inactive_days,sms_outgoing_to_onnet_count,sms_outgoing_to_onnet_spendings,sms_outgoing_to_onnet_inactive_days,sms_outgoing_to_offnet_count,sms_outgoing_to_offnet_spendings,sms_outgoing_to_offnet_i

**Užduotis**: iš čia naudoto kodo padaryti Apache Spark skriptą kuris nuskaito `customer_usage` failą, atlieka agregacijas pagal vartotojo id ir išsaugo rezultatą nurodytu keliu. Skipto parametrai turėtų būti bent du: 
1. įvesties failo kelias 
1. rezultatų išvesties kelias 